In [1]:
import tifffile
import numpy as np
import skimage.morphology as skm
import scipy.ndimage as sn

In [2]:
%matplotlib qt
import matplotlib.pyplot as plt

In [3]:
fn = '/NSLS2/xf18id1/users/2020Q1/XIAO_2020Q1/recon_fly_scan_id_57946/recon_fly_scan_id_57946_x358_y394_z470.tif'
imgs = tifffile.imread(fn)

In [113]:
img = imgs[0]
imgs_mask = np.ndarray(imgs.shape, dtype=np.int8)

In [116]:
thres = 3.6e-4
sigma=4
blur=12
for ii in range(imgs.shape[0]):
    imgs_mask[ii, :] = (sn.gaussian_filter(imgs[ii], sigma)>thres).astype(np.int8)[:]
    imgs_mask[ii, :] = skm.binary_dilation(imgs_mask[ii, :], selem=skm.disk(blur))[:]
    #print(ii)
print('done!')

done!


In [117]:
sfn = '/NSLS2/xf18id1/users/2020Q1/XIAO_2020Q1/recon_fly_scan_id_57946/recon_fly_scan_id_57946_x358_y394_z470_masked.tif'
tifffile.imsave(sfn, imgs*imgs_mask)

In [54]:
from ipywidgets import widgets
from ipywidgets import interact, fixed
from IPython.display import display, Image
import matplotlib as mpl
from io import BytesIO

In [110]:
def update_img(imgs, idx, sigma, thres, blur):
#    img = sn.gaussian_filter(imgs[idx], sigma)
    img_mask1 = (sn.gaussian_filter(imgs[idx], sigma)>thres).astype(np.int8)
    img_mask1 = skm.binary_dilation(img_mask1, selem=skm.disk(blur))
    with BytesIO() as buffer:
        mpl.image.imsave(buffer, imgs[idx]*img_mask1, format='jpg')
        out = buffer.getvalue()
#    with BytesIO() as buffer2:
#        mpl.image.imsave(buffer2, img, format='jpg')
#        out2 = buffer2.getvalue()
    return Image(out)

In [111]:
idx1 = (0, 470, 1)
idx2 = (1, 20, 1)
idx3 = (1e-5, 1e-3, 1e-5)
idx4 = (1, 20, 1)
w = widgets.interact(update_img, imgs=fixed(imgs), idx=idx1, sigma=idx2, thres=idx3, blur=idx4)
w.widget.children[2].readout_format = '.5f'


interactive(children=(IntSlider(value=235, description='idx', max=470), IntSlider(value=10, description='sigma…

In [109]:
help(skm.binary_dilation)

Help on function binary_dilation in module skimage.morphology.binary:

binary_dilation(image, selem=None, out=None)
    Return fast binary morphological dilation of an image.
    
    This function returns the same result as greyscale dilation but performs
    faster for binary images.
    
    Morphological dilation sets a pixel at ``(i,j)`` to the maximum over all
    pixels in the neighborhood centered at ``(i,j)``. Dilation enlarges bright
    regions and shrinks dark regions.
    
    Parameters
    ----------
    
    image : ndarray
        Binary input image.
    selem : ndarray, optional
        The neighborhood expressed as a 2-D array of 1's and 0's.
        If None, use a cross-shaped structuring element (connectivity=1).
    out : ndarray of bool, optional
        The array to store the result of the morphology. If None is
        passed, a new array will be allocated.
    
    Returns
    -------
    dilated : ndarray of bool or uint
        The result of the morphologica